In [22]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

170508288/170498071 [==============================] - 37s 0us/step


In [20]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [32]:
model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.L2(0.01)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    
    layers.Conv2D(64, 3, padding='same', kernel_regularizer=regularizers.L2(0.01)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    
    layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.L2(0.01)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    # layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_12 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 activation_12 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_13 (Bat  (None, 16, 16, 64)     

In [33]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10


2022-12-27 23:32:36.616036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 - 26s - loss: 1.9516 - accuracy: 0.5404 - 26s/epoch - 33ms/step
Epoch 2/10
782/782 - 25s - loss: 1.2779 - accuracy: 0.6536 - 25s/epoch - 32ms/step
Epoch 3/10
782/782 - 29s - loss: 1.2069 - accuracy: 0.6808 - 29s/epoch - 37ms/step
Epoch 4/10
782/782 - 27s - loss: 1.1689 - accuracy: 0.6947 - 27s/epoch - 34ms/step
Epoch 5/10
782/782 - 28s - loss: 1.1458 - accuracy: 0.7045 - 28s/epoch - 36ms/step
Epoch 6/10
782/782 - 27s - loss: 1.1248 - accuracy: 0.7126 - 27s/epoch - 35ms/step
Epoch 7/10
782/782 - 27s - loss: 1.1068 - accuracy: 0.7187 - 27s/epoch - 35ms/step
Epoch 8/10
782/782 - 27s - loss: 1.0878 - accuracy: 0.7287 - 27s/epoch - 34ms/step
Epoch 9/10
782/782 - 27s - loss: 1.0646 - accuracy: 0.7342 - 27s/epoch - 34ms/step
Epoch 10/10
782/782 - 26s - loss: 1.0558 - accuracy: 0.7377 - 26s/epoch - 34ms/step


2022-12-27 23:37:04.415288: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


157/157 - 2s - loss: 1.4431 - accuracy: 0.6122 - 2s/epoch - 15ms/step


[1.4430853128433228, 0.6122000217437744]